In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Create a dash application
app = dash.Dash(__name__)

In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',options=[
                                        {'label':'All Sites','value':'ALL'},
                                        {'label':'CCAFS LC-40','value':'CCAFS SLC-40'},
                                        {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                        {'label':'KSC LC-39A','value':'KSC LC-39A'}
                                    ],value='ALL',
                                    placeholder='Select a Launch Site here', 
                                    style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'
                                    },searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                marks={0: '0', 10000: '10000'},
                                                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
Input(component_id='site-dropdown', component_property='value'))

def load_pie_chart(site_dropdown):
    if site_dropdown == 'ALL':
        fig = px.pie(spacex_df,values='class',names="Launch Site",title='Total successfull launches')
        return fig
    else:
        df = spacex_df[spacex_df['Launch Site'] == site_dropdown]
        launch_site = df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(launch_site,values='class count',names='class',title=f'Success Launches for {site_dropdown}')
        return fig

In [6]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
[Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def load_scatter(site_dropdown,slider_range):
    low, high = slider_range
    df = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if site_dropdown == 'ALL':
        scatterTitle='Payload Mass vs. Outcome for All Launch Sites'
        fig = px.scatter(df,x='Payload Mass (kg)',y='class',color='Booster Version Category', title=scatterTitle)
        return fig
    else:
        df_fil = df[df['Launch Site']  == site_dropdown]
        scatterTitle=f'Payload mass vs. Outcome for Launch Site {site_dropdown}'
        fig = px.scatter(df_fil,x='Payload Mass (kg)',y='class',color='Booster Version Category', title=scatterTitle)
        return fig


In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Sep/2021 15:56:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 15:56:20] "GET /_dash-component-suites/dash/deps/polyfill@7.v1_21_0m1628533169.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 15:56:20] "GET /_dash-component-suites/dash/deps/prop-types@15.v1_21_0m1628533169.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 15:56:21] "GET /_dash-component-suites/dash/deps/react@16.v1_21_0m1628533169.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 15:56:21] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_17_1m1628533168.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 15:56:21] "GET /_dash-component-suites/dash/deps/react-dom@16.v1_21_0m1628533169.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 15:56:21] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_4m1628533166.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 15:56:21] "GE